# Compute daily average for estimation of TTDs using MESAS

This notebook converts the hourly data to daily data to use the MESAS package.

In [9]:
import numpy as np
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
sys.path.append('../../')
import WATRES
sites = ['Pully_small_storage','Pully_large_storage','Lugano_small_storage','Lugano_large_storage','Basel_small_storage','Basel_large_storage']

def convert_data4MESAS(site):
    df = pd.read_csv('../{0}/data/{0}.txt'.format(site), sep='\t')
    df['datetime'] = df['t'].apply(lambda x: WATRES.fractional_year_to_datetime(x))
    # Step 1: Convert the 'datetime' column to datetime format
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d-%H:%M:%S')


    # Calculate the quartiles of the 'q' column
    q_quantiles = df['q'][-365*24:].quantile([0.25, 0.5, 0.75]).to_dict() 
    for key, value in q_quantiles.items():
        q_quantiles[key] = value * 24
    
    # Step 2: Set 'datetime' as the index for easier resampling
    df.set_index('datetime', inplace=True)
    
    # Step 3: Resample the data to daily frequency and aggregate
    # For columns 'p', 'pet', 'q', we compute the sum for each day
    daily_data = df.resample('D').agg({
        'p': 'sum',
        'pet': 'sum',
        'et': 'sum',
        'q': 'sum'
    })
    
    # For columns 'Cp' and 'Cq', compute the weighted average where the weights are given by 'p'
    daily_data['Cp'] = df['Cp'].resample('D').apply(lambda x: (x * df.loc[x.index, 'p']).sum() / df.loc[x.index, 'p'].sum())
    daily_data['Cq'] = df['Cq'].resample('D').apply(lambda x: (x * df.loc[x.index, 'q']).sum() / df.loc[x.index, 'q'].sum())
    
    # Reset index if you want 'datetime' to be a regular column
    daily_data.reset_index(inplace=True)


    
    # Add columns for each quartile with 0 or 1 depending on which quartile the 'q' value belongs to
    daily_data['Q_quantile_0'] = (daily_data['q'] <= q_quantiles[0.25]).astype(int)
    daily_data['Q_quantile_1'] = ((daily_data['q'] > q_quantiles[0.25]) & (daily_data['q'] <= q_quantiles[0.5])).astype(int)
    daily_data['Q_quantile_2'] = ((daily_data['q'] > q_quantiles[0.5]) & (daily_data['q'] <= q_quantiles[0.75])).astype(int)
    daily_data['Q_quantile_3'] = (daily_data['q'] > q_quantiles[0.75]).astype(int)


    # Now, daily_data contains the daily sum for p, pet, q, and the weighted averages for Cp and Cq.
    daily_data.to_csv('../{0}/data/{0}_daily.txt'.format(site), index=False)

In [10]:
for site in sites:
    convert_data4MESAS(site)

/tmp/ipykernel_1873/3575359864.py:35: RuntimeWarning: invalid value encountered in scalar divide
  daily_data['Cp'] = df['Cp'].resample('D').apply(lambda x: (x * df.loc[x.index, 'p']).sum() / df.loc[x.index, 'p'].sum())
/tmp/ipykernel_1873/3575359864.py:35: RuntimeWarning: invalid value encountered in scalar divide
  daily_data['Cp'] = df['Cp'].resample('D').apply(lambda x: (x * df.loc[x.index, 'p']).sum() / df.loc[x.index, 'p'].sum())
/tmp/ipykernel_1873/3575359864.py:35: RuntimeWarning: invalid value encountered in scalar divide
  daily_data['Cp'] = df['Cp'].resample('D').apply(lambda x: (x * df.loc[x.index, 'p']).sum() / df.loc[x.index, 'p'].sum())
/tmp/ipykernel_1873/3575359864.py:35: RuntimeWarning: invalid value encountered in scalar divide
  daily_data['Cp'] = df['Cp'].resample('D').apply(lambda x: (x * df.loc[x.index, 'p']).sum() / df.loc[x.index, 'p'].sum())
/tmp/ipykernel_1873/3575359864.py:35: RuntimeWarning: invalid value encountered in scalar divide
  daily_data['Cp'] = df